<a href="https://colab.research.google.com/github/IKAR42/SberDS_Study/blob/main/Final_pet_project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ТЗ по старту проекта - шаги:

1. Провести EDA по обоим Data сетам
  * 1.1 Проверяем на наличие пропусков, аномалий - смотрим статистики по дата 
сету 
  * Смотрим распределения
  * Чистим данные +
  * Строим графики 
    * умозаключение для каждого графика
  * Смотрим сегменты пользователей + график (определяем основные параметры кластера)
  * Попробовать посмотреть причастность unknown к одной из групп
  * Пишем выводы

2. Реализация основной задачи
  * Изучить вопрос 
      * коллаборативная фильтрация
      * factorisation machine
  * В случае реализации нескольких подходов определить какой лучше подходит

3. Подготовка презентационных материалов

##Известная информация на момент старта проекта:
* Блок дополнительно необязателен для выполнения проектной задачи
* Присутствующие в датасете unknown обработать (допускаем исключение из дата сета, либо OHE)

* При взаимном лайке возникает возможность общения
Для упращения можно допустить: если rating >=6, то это лайк, иначе - дизлайк

* Необходимо написать код, который поможет найти для следуюших пользователей, которые они с максимальной вероятностью залайкают (за вычетом известных в датасете пар). Для каждого id - не более 10 пользователей

id: 98447, 111944, 111930, 82091, 125299, 125298, 17335, 60356

* Дополнительно:
Необходимо сформировать 2 колоды пользователей для каждого:
  * Те, которых пользователь с максимальной вероятностью лайкнет
  * Те, с кем максимальной вероятностью будет мэтч (взаимный лайк)
  * Упаковать в сервис по API (flask)

In [ ]:
# Материалы, потом убрать
https://makesomecode.me/2018/09/movie-recommendation-system/

https://www.kaggle.com/code/methoomirza/books-recomender-popular-collaborative-with-webapp

https://stackru.com/questions/52338451/raschet-kosinusnyih-rasstoyanij-dlya-frejma-dannyih-pandas (со спарком)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка всех необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np

Подключение Google диска 

## 1.Подготовка дата сета

In [ ]:
# Дата сет 1. Пользователь с id_from оценил (rating) пользователя id_to

df_ratings = pd.read_csv('drive/My Drive/rec-libimseti-dir-another.edges', delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])

In [ ]:
df_ratings.head(5)

,id_from,id_to,rating
0,1,133,8
1,1,720,6
2,1,971,10
3,1,1095,7
4,1,1616,10


In [ ]:
# В датасете данные о пользователе 
# 1ая колонка id
# 2ая колонка gender: "M" - male/мужчина, "F" - female/женщина,  "U" - unknown
# по остальным колонкам недоступна информация

df_info = pd.read_csv('drive/My Drive/info.csv')

In [ ]:
df_info.head()

,id,gender,1,2,3,4,5,6,7,8,9,10,11,12
0,1,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,2,F,1,46,81,6,9,31,22,79,82,75,53,0.675761
2,3,U,4,69,64,2,18,44,54,80,50,89,38,0.927339
3,4,F,3,46,90,9,42,54,59,86,60,65,25,0.176650
4,5,F,3,79,83,1,45,57,1,58,34,23,33,0.553003


In [ ]:
# Смотрим базовые статистики по дата сету
print('='*20, ' INFO ', '='*20)
print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings.info())

print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings.columns)

print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings.shape)

print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings.describe())

print('\n', '-'*55)

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info.info())

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info.columns)

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info.shape)

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info.describe())

====================  INFO  ====================

 -------------------- 1. Ratings  --------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17359346 entries, 0 to 17359345
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   id_from  int64
 1   id_to    int64
 2   rating   int64
dtypes: int64(3)
memory usage: 397.3 MB
None

 -------------------- 1. Ratings  --------------------
Index(['id_from', 'id_to', 'rating'], dtype='object')

 -------------------- 1. Ratings  --------------------
(17359346, 3)

 -------------------- 1. Ratings  --------------------
            id_from         id_to        rating
count  1.735935e+07  1.735935e+07  1.735935e+07
mean   6.787292e+04  1.013644e+05  5.938437e+00
std    3.876211e+04  6.218369e+04  3.112005e+00
min    1.000000e+00  1.000000e+00  1.000000e+00
25%    3.465000e+04  4.813900e+04  3.000000e+00
50%    6.792700e+04  9.731500e+04  6.000000e+00
75%    1.010250e+05  1.513820e+05  9.000000e+00
max    1.353590e

In [ ]:
# Смотрим пропущенные значения
print('='*20, ' MISSING VALUES ', '='*20)
print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings.isna().sum() )

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info.isna().sum() )

====================  MISSING VALUES  ====================

 -------------------- 1. Ratings  --------------------
id_from    0
id_to      0
rating     0
dtype: int64

 -------------------- 2. Users  --------------------
id         0
 gender    0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
dtype: int64


In [ ]:
# Удаляем дубликаты
df_ratings_new = df_ratings.drop_duplicates()
df_info_new = df_info.drop_duplicates()

In [ ]:
# Проверяем обьем дата сета после удаления дубликатов
# В результате обьем выборки не изменился
print('='*20, ' CHECK DUPLICATES ', '='*20)
print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings_new.info())

print('\n', '-'*20, '1. Ratings ', '-'*20)
print(df_ratings_new.shape)

print('\n', '-'*55)

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info_new.info())

print('\n', '-'*20, '2. Users ', '-'*20)
print(df_info_new.shape)


====================  CHECK DUPLICATES  ====================

 -------------------- 1. Ratings  --------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17359346 entries, 0 to 17359345
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   id_from  int64
 1   id_to    int64
 2   rating   int64
dtypes: int64(3)
memory usage: 529.8 MB
None

 -------------------- 1. Ratings  --------------------
(17359346, 3)

 -------------------------------------------------------

 -------------------- 2. Users  --------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 220970 entries, 0 to 220969
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       220970 non-null  int64  
 1    gender  220970 non-null  object 
 2   1        220970 non-null  int64  
 3   2        220970 non-null  int64  
 4   3        220970 non-null  int64  
 5   4        220970 non-null  int64  
 6   5        22097

In [ ]:
#Для полноценного решения задачи требуется обьединить эти 2 дата сета
df_ratings_new = df_ratings_new.rename(columns={'id_from': 'id'})

In [ ]:
Users_rating = df_ratings_new.merge(df_info_new, on = 'id')
print("\nShape: ", Users_rating.shape )


Shape:  (17359346, 16)


In [ ]:
Users_rating.head()

,id,id_to,rating,gender,1,2,3,4,5,6,7,8,9,10,11,12
0,1,133,8,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,1,720,6,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
2,1,971,10,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
3,1,1095,7,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
4,1,1616,10,F,1,45,90,15,8,13,63,57,96,42,87,0.934487


In [ ]:
# Смотрим базовые статистики по дата сету
# Все в норме, столбцы и строки не потерялись
print('='*20, ' INFO ', '='*20)
print('\n', '-'*20, 'Merged_data ', '-'*20)
print(Users_rating.info())

print('\n', '-'*20, 'Merged_data ', '-'*20)
print(Users_rating.columns)

print('\n', '-'*20, 'Merged_data ', '-'*20)
print(Users_rating.shape)

====================  INFO  ====================

 -------------------- Merged_data  --------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17359346 entries, 0 to 17359345
Data columns (total 16 columns):
 #   Column   Dtype  
---  ------   -----  
 0   id       int64  
 1   id_to    int64  
 2   rating   int64  
 3    gender  object 
 4   1        int64  
 5   2        int64  
 6   3        int64  
 7   4        int64  
 8   5        int64  
 9   6        int64  
 10  7        int64  
 11  8        int64  
 12  9        int64  
 13  10       int64  
 14  11       int64  
 15  12       float64
dtypes: float64(1), int64(14), object(1)
memory usage: 2.2+ GB
None

 -------------------- Merged_data  --------------------
Index(['id', 'id_to', 'rating', ' gender', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12'],
      dtype='object')

 -------------------- Merged_data  --------------------
(17359346, 16)


In [ ]:
# Удаляем дубликаты
Users_rating_new = Users_rating.drop_duplicates()

In [ ]:
# Проверяем уникальные значения в дата сете
Users_rating_new.nunique()

id         135359
id_to      168791
rating         10
 gender         3
1               5
2              72
3              99
4              24
5              60
6              60
7             100
8             100
9             100
10            100
11            100
12         135359
dtype: int64

----------------------------------- Итоги -----------------------------------

Предварительная подготовка данных окончена:

1. Для работы есть 2 дата сета
2. В дата сете df_ratings
    *   Пользователь с id_from оценил (rating) пользователя id_to
3. В дата сете df_info
    *   1ая колонка id
    *   2ая колонка gender: "M" - male/мужчина, "F" - female/женщина,  "U" - unknown
    *   По остальным колонкам информация недоступна
4. Для решения задачи требуется обьединить эти дата сеты в 1 по столбцу id
5. После обьединения данные утеряны не были. Дубликаты не обнаружены





## 2. EDA дата сета

In [ ]:
# Посчитать количество пользователей по полу 
# Посмотреть распределение где x - пол, y - оценка (для каждого рейтинга,пола своя) - визуализировать
# Посмотреть максимальное, минимальное количество оценок для каждого пола (по gender + id агрегат) - визуализировать
# Посмотреть что делать с U (обьем данных)
# Кластеризовать пользователей в дата сете -> определить средние параметры каждого кластера

In [ ]:
# Проверяем уникальные значения столбца с полом юзеров
Users_rating_new[' gender'].unique()

array(['F', 'U', 'M'], dtype=object)

In [ ]:
Users_rating_count = Users_rating_new.groupby(' gender')['rating'].count().sort_values(ascending=True).reset_index()
Users_rating_count

,gender,rating
0,U,1702845
1,M,4852458
2,F,10804043


In [ ]:
Users_rating_new.shape

(17359346, 16)

### 1.2 MVP

In [ ]:
# Как идея, прежде чем конкатить, использовать все параметры и сам рейтинг = придумать свой суммарный параметр, которому можно понизить размерность
  # Это позволит уменьшить потребление ОЗУ

In [ ]:
Users_rating_small_1 = Users_rating_new[:50000]

In [ ]:
Users_rating_small_1

,id,id_to,rating,gender,1,2,3,4,5,6,7,8,9,10,11,12
0,1,133,8,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,1,720,6,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
2,1,971,10,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
3,1,1095,7,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
4,1,1616,10,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,325,66444,6,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
49996,325,66446,1,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
49997,325,66463,1,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
49998,325,66663,5,M,5,30,14,1,5,4,74,75,53,33,78,0.919760


In [ ]:
pivot_rating_1 = Users_rating_small.pivot_table(index = 'id_to', columns = 'id', values = 'rating')

In [ ]:
pivot_rating_1

id,1,2,3,4,5,6,7,8,9,10,...,316,317,318,319,320,321,322,323,324,325
id_to,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pivot_rating_1.fillna(0, inplace=True)
pivot_rating_1

id,1,2,3,4,5,6,7,8,9,10,...,316,317,318,319,320,321,322,323,324,325
id_to,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pivot_rating_1)

In [ ]:
cosine_sim_df_2 = pd.DataFrame(similarity_score)
cosine_sim_df_2

,0,1,2,3,4,5,6,7,8,9,...,27032,27033,27034,27035,27036,27037,27038,27039,27040,27041
0,1.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.707107,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.816497,0.0,0.000000
1,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.275535
2,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.502519,0.000000,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.408248,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27037,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000
27038,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000
27039,0.816497,0.000000,0.0,0.0,0.408248,0.0,0.0,0.577350,0.410305,0.112747,...,0.408248,0.000000,0.354459,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000
27040,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.640184,0.000000,0.0,0.0,0.0,0.0,0.000000,1.0,0.000000


In [ ]:
print("Shape: ", cosine_sim_df_2.shape)
similarity_score

Shape:  (27042, 27042)


array([[1.        , 0.        , 0.        , ..., 0.81649658, 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.27553464],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.81649658, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.27553464, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
sorted(list(enumerate(similarity_score[5])), key=lambda x:x[1], reverse=True)[1:6]

[(199, 1.0), (310, 1.0), (397, 1.0), (550, 1.0), (883, 1.0)]

In [ ]:
np.where(pivot_rating_1.iloc[505])[0][0]

274

In [ ]:
pivot_rating_1.index

Int64Index([     1,      2,     17,     18,     35,     38,     41,     42,
                55,     57,
            ...
            220837, 220840, 220848, 220859, 220861, 220883, 220900, 220916,
            220948, 220953],
           dtype='int64', name='id_to', length=27042)

In [ ]:
def recommender(man_id):
    indx = np.where(pivot_rating_1.iloc[man_id])[0][0]
    similar_books = sorted(list(enumerate(similarity_score[indx])), key=lambda x:x[1], reverse=True)[1:15]
    data = []
    for i in similar_books:
        item = []
        temp_df = Users_rating_small[Users_rating_small['id_to'] == pivot_rating_1.index[i[0]] ]
        item.extend(list(temp_df.drop_duplicates('id_to')['id_to'].values))
        item.extend(list(temp_df.drop_duplicates('id_to')['rating'].values ))
        item.extend(list(temp_df.drop_duplicates('id_to')[' gender'].values ))          
        data.append(item)
    
    return data

In [ ]:
#98447, 111944, 111930, 82091, 125299, 125298, 17335, 60356

recommender(1313)

[[46854, 5, 'M'],
 [27963, 6, 'M'],
 [35856, 2, 'M'],
 [116133, 4, 'M'],
 [55661, 5, 'M'],
 [88730, 6, 'M'],
 [23900, 4, 'M'],
 [41302, 3, 'M'],
 [11129, 9, 'M'],
 [30709, 7, 'M'],
 [65445, 6, 'M'],
 [31398, 7, 'M'],
 [178235, 6, 'M'],
 [65808, 4, 'M']]

In [ ]:
Users_rating_small[Users_rating_small['id'] == '46854']

,id,id_to,rating,gender,1,2,3,4,5,6,7,8,9,10,11,12


In [ ]:
Users_rating_small.iloc[1313]

id               9
id_to        21767
rating           9
 gender          M
1                4
2               82
3                8
4                3
5               38
6               57
7               63
8               83
9               72
10              89
11              92
12         0.24699
Name: 1313, dtype: object

In [ ]:
# по идее сама рекомендация лежит здесь в id_to
Users_rating_small.iloc[46854]

id              316
id_to          1893
rating            4
 gender           M
1                 5
2                54
3                48
4                15
5                41
6                51
7                33
8                57
9                42
10               64
11               98
12         0.314681
Name: 46854, dtype: object

In [ ]:
Users_rating_small

,id,id_to,rating,gender,1,2,3,4,5,6,7,8,9,10,11,12
0,1,133,8,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,1,720,6,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
2,1,971,10,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
3,1,1095,7,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
4,1,1616,10,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,325,66444,6,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
49996,325,66446,1,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
49997,325,66463,1,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
49998,325,66663,5,M,5,30,14,1,5,4,74,75,53,33,78,0.919760
